<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/3%20Aprendizaje%20profundo%20(II)/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torchvision
from torchvision import transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, Subset
from torch import nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

We are going to create a GAN to generate files that are similar to Bob Ross

In [ ]:
# Access to the BOB ROSS images folder
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!ls gdrive/MyDrive/UNIVERSIDAD/Master\ Propio\ IA\ UV/3\ Aprendizaje\ profundo\ (II)/Datasets/Bob\ Ross/train/images
directory = 'gdrive/MyDrive/UNIVERSIDAD/Master Propio IA UV/3 Aprendizaje profundo (II)/Datasets/Bob Ross/train/images'

In [ ]:
transform = transforms.Compose([
  transforms.ToTensor()
])

In [ ]:
directory = "gdrive/MyDrive/UNIVERSIDAD/Master Propio IA UV/3 Aprendizaje profundo (II)/Datasets/Bob Ross/train"

In [ ]:
folder = datasets.ImageFolder(root = directory, transform=transform)

In [ ]:
folder

Dataset ImageFolder
    Number of datapoints: 500
    Root location: gdrive/MyDrive/UNIVERSIDAD/Master Propio IA UV/3 Aprendizaje profundo (II)/Datasets/Bob Ross/train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

In [ ]:
batch_size = 20

folder_loader = DataLoader(folder, batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
def train_val_dataset(dataset, val_split=0.25):
  train_idx, val_idx = train_test_split(list(range(len(dataset))), test_size=val_split, shuffle=True)
  datasets = {}
  datasets['train'] = Subset(dataset, train_idx)
  datasets['val'] = Subset(dataset, val_idx)
  return datasets

In [ ]:
datasets = train_val_dataset(folder)

### Small experiment to check how grad works in PyTorch tensors

In [ ]:
x = torch.randn(5)
w = torch.randn(3,5).requires_grad_()
b = torch.randn(3).requires_grad_()

y = torch.randn(3)

In [ ]:
def linear(x, w, b):
  return x @ w.t() + b

def mse(pred, real):
  return (pred - real).pow(2).mean()

In [ ]:
result = linear(x, w, b)
loss = mse(result, y)

In [ ]:
loss

tensor(5.4519, grad_fn=<MeanBackward0>)

In [ ]:
loss.backward()

In [ ]:
w.grad

tensor([[ 0.9431, -0.0135, -0.7908,  0.0409,  0.9231],
        [ 1.6658, -0.0239, -1.3967,  0.0723,  1.6304],
        [ 3.4015, -0.0488, -2.8520,  0.1475,  3.3292]])

In [ ]:
w.data

tensor([[ 1.2697, -0.6939,  0.1321, -0.1466, -0.3914],
        [-1.2386, -1.8907, -1.8679, -1.9964,  0.7666],
        [ 0.6081,  0.5699,  0.1135, -0.5932,  0.5787]])

In [ ]:
w

tensor([[ 1.2697, -0.6939,  0.1321, -0.1466, -0.3914],
        [-1.2386, -1.8907, -1.8679, -1.9964,  0.7666],
        [ 0.6081,  0.5699,  0.1135, -0.5932,  0.5787]], requires_grad=True)

In [ ]:
b.data

tensor([ 0.5760,  0.7353, -1.4715])

In [ ]:
b.grad

tensor([-0.6515, -1.1507, -2.3496])

In [ ]:
a1 = torch.randn(1,3).requires_grad_()
a1.shape

torch.Size([1, 3])

In [ ]:
aa1 = a1.squeeze()
aa1.shape

torch.Size([3])

In [ ]:
a2 = torch.randn(2,3).requires_grad_()
a2.shape

torch.Size([2, 3])

In [ ]:
aa2 = a2.squeeze()
aa2.shape #The squeeze function does nothing because there is no dimension to squeeze

torch.Size([2, 3])

### Continuing with building the Bob Ross generator

In [ ]:
##Right now, the generator only generates one image per execution,
## maybe it should be readapted to generate a BATCH of images

class Generator(nn.Module):
  def __init__(self, noise_size, image_size):
    super.__init__()

    self.hidden_channels = 10
    self.output_channels = 3
    self.hidden_size = self.hidden_channels * image_size ** 2

    self.linear = nn.Linear(noise_size, hidden_size)
    self.conv1 = nn.ConvTranspose2d(self.hidden_channels, self.hidden_channels//2, (3,3))
    self.conv2 = nn.ConvTranspose2d(self.hidden_channels//2, self.output_channels, (3,3))



  def forward(self, x):
    x = F.relu(self.linear(x))
    x = x.view(image_size, image_size, -1)
    x = F.relu(F.upsample_bilinear(self.conv1(self.conv1(x)))) ##THIS IS NOT GOING TO WORK. THE CHANNELS DON'T MATCH
    x = F.relu(F.upsample_bilinear(self.conv2(self.conv2(x))))


In [ ]:
class Discriminator(nn.Module):
  def __init__(self, image_size):
    super.__init__()

    self.im_width, self.im_height, self.im_channels = image_size

    self.conv1_1 = nn.Conv2d(self.im_channels, 16, (3,3))
    self.conv1_2 = nn.Conv2d(16, 16, (3,3))
    self.conv2_1 = nn.Conv2d(16, 32, (3,3))
    self.conv2_2 = nn.Conv2d(32, 32, (3,3))
    self.conv3_1 = nn.Conv2d(32, 64, (3,3))
    self.conv3_2 = nn.Conv2d(64, 64, (3,3))
    self.pool = nn.MaxPool2d(kernel_size=(2,2))

    self.dense1 = nn.Linear(128, 32)
    self.dense2 = nn.Linear(32, 2)

  def forward(self, x):
    x = F.relu(self_conv1_1(x))
    x = F.relu(self_conv1_2(x))
    x = F.batch_norm(self.pool(x))
    x = F.relu(self.conv2_1(x))
    x = F.relu(self.conv2_2(x))
    x = F.batch_norm(self.pool(x))
    x = F.relu(self.conv3_1(x))
    x = F.relu(self.conv3_2(x))
    x = F.batch_norm(self.pool(x))
    x = nn.Flatten(x)
    x = F.relu(self.dense1(x))
    x = F.softmax(self.dense2(x))

In [4]:
x = torch.randn(5, 32, 32, 3)
m = F.avg_pool2d(kernel_size=)
y = F.avg_pool2d(x)
y.shape

TypeError: ignored